# SQL ASSIGNMENT

## Python Code

In [62]:
# Install dependencies
!pip install faker pandas sqlite-utils


## Import required libraries

In [63]:
#import sqlite3
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta


In [64]:
# Initialize Faker
fake = Faker()


In [65]:
# Blood types and specializations
BLOOD_TYPES = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']
SPECIALIZATIONS = ['Cardiology', 'Neurology', 'Orthopedics', 'Pediatrics', 'General Medicine']


In [66]:
# Define number of records
NUM_PATIENTS = 1000
NUM_DOCTORS = 50
NUM_MEDICAL_RECORDS = 1500
NUM_APPOINTMENTS = 1200
NUM_BILLINGS = 1000


In [67]:
# Create lists for each table's data
patients = []
doctors = []
medical_records = []
appointments = []
billings = []


## Generate Patient Data

In [68]:
# Generate patients data (with some duplicate and missing data)
for i in range(NUM_PATIENTS):
    name = fake.name() if random.random() > 0.05 else None  # 5% missing names
    age = random.randint(1, 100)  # Realistic age range
    gender = random.choice(['Male', 'Female', 'Other'])
    blood_type = random.choice(BLOOD_TYPES)
    contact = fake.phone_number() if random.random() > 0.1 else None  # 10% missing contacts
    address = fake.address()
    insurance = fake.company() if random.random() > 0.2 else None  # 20% missing insurance
    patients.append([i + 1, name, age, gender, blood_type, contact, address, insurance])

# Add some duplicate patient data
for _ in range(5):
    duplicate_patient = random.choice(patients[:-5])  # Select a random existing patient
    patients.append(duplicate_patient)  # Add duplicate entry

print(f"{NUM_PATIENTS + 5} patients generated (including 5 duplicates) with realistic missing data!")


1005 patients generated (including 5 duplicates) with realistic missing data!


## Generate Doctor Data

In [69]:
# Generate doctors data
for i in range(NUM_DOCTORS):
    doctors.append([i + 1, fake.name(), random.choice(SPECIALIZATIONS), random.randint(1, 40), fake.phone_number()])

print(f"{NUM_DOCTORS} doctors generated with random specializations and experience.")


50 doctors generated with random specializations and experience.


## Generate Medical Data

In [70]:
# Generate medical records
for i in range(NUM_MEDICAL_RECORDS):
    patient_id = random.randint(1, NUM_PATIENTS)
    doctor_id = random.randint(1, NUM_DOCTORS)
    diagnosis = random.choice(['Flu', 'Diabetes', 'Hypertension', 'COVID-19', 'Asthma', 'Cancer'])
    treatment = random.choice(['Medication', 'Surgery', 'Therapy', 'Observation', 'Hospitalization'])
    admission_date = fake.date_between(start_date="-2y", end_date="today")
    discharge_date = admission_date + timedelta(days=random.randint(1, 10))
    medical_records.append([i + 1, patient_id, diagnosis, treatment, admission_date, discharge_date, doctor_id])

print(f"{NUM_MEDICAL_RECORDS} medical records generated with random diagnoses, treatments, and dates.")


1500 medical records generated with random diagnoses, treatments, and dates.


## Generate Appointment Data

In [71]:
# Generate appointments
for i in range(NUM_APPOINTMENTS):
    appointments.append([i + 1, random.randint(1, NUM_PATIENTS), random.randint(1, NUM_DOCTORS),
                         fake.date_between(start_date="-1y", end_date="today"),
                         random.choice(['Scheduled', 'Completed', 'Canceled'])])

print(f"{NUM_APPOINTMENTS} appointments generated with random patient-doctor pairs and appointment statuses.")


1200 appointments generated with random patient-doctor pairs and appointment statuses.


## Generate Billing Data

In [72]:
# Generate billing data
for i in range(NUM_BILLINGS):
    billings.append([i + 1, random.randint(1, NUM_PATIENTS), round(random.uniform(100, 5000), 2),
                     random.choice(['Paid', 'Unpaid', 'Pending']),
                     random.choice(['Approved', 'Rejected', 'Processing'])])

print(f"{NUM_BILLINGS} billing records generated with random amounts and payment statuses.")


1000 billing records generated with random amounts and payment statuses.


## Convert to DataFrames

In [73]:
# Convert to DataFrames after data is generated
patients_df = pd.DataFrame(patients, columns=['patient_id', 'name', 'age', 'gender', 'blood_type', 'contact_number', 'address', 'insurance_provider'])
doctors_df = pd.DataFrame(doctors, columns=['doctor_id', 'name', 'specialization', 'experience_years', 'contact_number'])
medical_records_df = pd.DataFrame(medical_records, columns=['record_id', 'patient_id', 'diagnosis', 'treatment', 'admission_date', 'discharge_date', 'doctor_id'])
appointments_df = pd.DataFrame(appointments, columns=['appointment_id', 'patient_id', 'doctor_id', 'appointment_date', 'status'])
billings_df = pd.DataFrame(billings, columns=['bill_id', 'patient_id', 'amount_due', 'payment_status', 'insurance_claim_status'])


## SAVE as CSV Files

In [74]:
# Save as CSV files
patients_df.to_csv("patients.csv", index=False)
doctors_df.to_csv("doctors.csv", index=False)
medical_records_df.to_csv("medical_records.csv", index=False)
appointments_df.to_csv("appointments.csv", index=False)
billings_df.to_csv("billings.csv", index=False)

print("CSV files created successfully. Now, upload them to SQLite.")


CSV files created successfully. Now, upload them to SQLite.


# SQL CODE

In [75]:
# Import required libraries
import sqlite3
import graphviz


## Connect to a SQLite Database

In [76]:
# Connect to SQLite database
conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()


## Create the tables in SQLite Database

###  Table 1 : Patients

In [77]:
# Create a Patient table into SQLite Database.
cursor.execute('''
CREATE TABLE IF NOT EXISTS Patients (
    patient_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    gender TEXT CHECK(gender IN ('Male', 'Female', 'Other')),
    blood_type TEXT,
    contact_number TEXT,
    address TEXT,
    insurance_provider TEXT
);
''')


###  Table 2 : Doctors

In [78]:
# Create a Doctors table into SQLite Database.
cursor.execute('''
CREATE TABLE IF NOT EXISTS Doctors (
    doctor_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    specialization TEXT,
    experience_years INTEGER CHECK(experience_years >= 0),
    contact_number TEXT UNIQUE
);
''')


###  Table 3 : Medical_Records

In [79]:
# Create a Medical_Records table into SQLite Database.
cursor.execute("""
CREATE TABLE IF NOT EXISTS Medical_Records (
    record_id INTEGER PRIMARY KEY,
    patient_id INTEGER,
    diagnosis TEXT,
    treatment TEXT,
    admission_date DATE,
    discharge_date DATE,
    doctor_id INTEGER,
    FOREIGN KEY (patient_id) REFERENCES Patients(patient_id),
    FOREIGN KEY (doctor_id) REFERENCES Doctors(doctor_id)
);
""")


###  Table 4 : Appointments

In [80]:
# Create a Appointments table into SQLite Database.
cursor.execute("""
CREATE TABLE IF NOT EXISTS Appointments (
    appointment_id INTEGER PRIMARY KEY,
    patient_id INTEGER,
    doctor_id INTEGER,
    appointment_date DATE,
    status TEXT CHECK(status IN ('Scheduled', 'Completed', 'Canceled')),
    FOREIGN KEY (patient_id) REFERENCES Patients(patient_id),
    FOREIGN KEY (doctor_id) REFERENCES Doctors(doctor_id),
    UNIQUE (patient_id, doctor_id, appointment_date)
);
""")


###  Table 5 : Billings

In [81]:
# Create a Billings table into SQLite Database.
cursor.execute("""
CREATE TABLE IF NOT EXISTS Billings (
    bill_id INTEGER PRIMARY KEY,
    patient_id INTEGER,
    amount_due REAL CHECK(amount_due > 0),
    payment_status TEXT CHECK(payment_status IN ('Paid', 'Unpaid', 'Pending')),
    insurance_claim_status TEXT CHECK(insurance_claim_status IN ('Approved', 'Rejected', 'Processing')),
    FOREIGN KEY (patient_id) REFERENCES Patients(patient_id)
);
""")


## Load CSV into SQLite

In [82]:
patients_df.to_sql('Patients', conn, if_exists='replace', index=False)
doctors_df.to_sql('Doctors', conn, if_exists='replace', index=False)
medical_records_df.to_sql('Medical_Records', conn, if_exists='replace', index=False)
appointments_df.to_sql('Appointments', conn, if_exists='replace', index=False)
billings_df.to_sql('Billings', conn, if_exists='replace', index=False)


1000

In [83]:
# Query to check data
df = pd.read_sql_query("SELECT * FROM Patients LIMIT 5", conn)
print(df)


   patient_id             name  age gender blood_type        contact_number  \
0           1       Lynn Clark   81  Other         O+      001-678-623-9345   
1           2             None   28   Male         O-  001-580-902-5699x076   
2           3     Mary Johnson   81  Other         O+          508.987.1634   
3           4  Jennifer Howard   99   Male        AB-                  None   
4           5      Chad Guzman   68   Male         B+   +1-233-777-4517x523   

                                             address  \
0          98238 Stevens Light\nMontesland, KY 00597   
1        7496 Clark Turnpike\nRobinsonfurt, GA 11707   
2          7541 Wesley Rest\nWest Tinaland, PA 05987   
3  141 Barber Wells Apt. 598\nSouth Kristen, MD 7...   
4  42950 Madison Cliff Suite 945\nChristinaview, ...   

               insurance_provider  
0                            None  
1      Fischer, Dalton and Huerta  
2                     Perry-Crane  
3  Mendoza, Rodriguez and Collins  
4       

## Generate a .png image of the database schema, where each table is shown as a node and each column is represented as an edge.

In [84]:
# Retrieve Table and Column Information
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Create a Graphviz Digraph
dot = graphviz.Digraph(format='png')

# For each table, add it to the diagram
for table in tables:
    table_name = table[0]
    dot.node(table_name, table_name)

    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()

    # Add columns to the table node
    for column in columns:
        column_name = column[1]
        dot.edge(table_name, column_name, label=column_name)

# Render the Graph
dot.render('db_schema')


'db_schema.png'

## Close a SQLite Database Connection

In [85]:
conn.close()
